In [1]:
"""
    author: Serena Shen
    @ 2017-09-16
"""

In [2]:
import tensorflow as tf
import numpy as np

In [10]:
# 定义数据流图dataflow
a = tf.constant(5, name='input_a')
b = tf.constant(3, name='input_b')
c = a+b
d = a*b
e = c+d

In [11]:
# 创建TensorFlow session对象
sess = tf.Session()
output = sess.run(e)

In [12]:
# 创建writer对象
writer = tf.summary.FileWriter('./my_graph', sess.graph)

# cmd-> cd path ->tensorboard --logdir='my_graph'

In [15]:
# 关闭session,writer对象
writer.close()
sess.close()

In [25]:
# 张量
a = tf.constant([[5, 3, 2, 1], [4, 3, 2, 1]], name='input_a')
b = tf.reduce_prod(a, name='prod_b', axis=1)
c = tf.reduce_sum(a, name='sum_c', axis=1)
e = c+b

sess = tf.Session()
sess.run(e)

array([41, 34])

In [ ]:
# Graph 对象

In [27]:
## 创建流图
# 初始化dataflow
g = tf.Graph()
# 添加operation
with g.as_default():
    a = tf.constant(5)
    
# 不在with语句块中的定义op都在默认的dataflow graph里
default_graph = tf.get_default_graph()  # 获取默认的graph

array([30, 24])

In [ ]:
# Session 对象
# Session.run()接收一个参数fetches，以及其他三个可选参数：feed_dict\options & run_metadata

In [ ]:
sess.run([a, b])     # Op 列表
sess.run(tf.initialize_all_variable())  # 执行初始化Variable对象，返回值是None

In [ ]:
# feed_dict 参数 用于覆盖tensor对象的值 要求传入字典
a = tf.constant(2)
b = tf.constant(3)
c = a+b
d = a*b
e = c+d
with tf.Session() as sess:
    print(sess.run(e, feed_dict={d: 5}))
    
# 推荐下面的做法    
sess = tf.Session()
with sess.as_default():
    print(e.eval())     # sess.run(e)

10
11


In [ ]:
# placeholder 利用占位节点添加输入
# dtype参数是必须的
a = tf.placeholder(tf.int32, shape=[2], name='input_a')
b = tf.reduce_sum(a, name='sum_b')
c = tf.reduce_prod(a, name='prod_c')
d = b+c
sess = tf.Session()
input_dict = {a: np.array([5, 3], dtype=np.int32)}
sess.run(d, feed_dict=input_dict)

In [10]:
# Variable
# 1.创建Variable对象
var1 = tf.Variable(3, name='var1', trainable=False)
sess = tf.Session()
# 用于提供初始化值的辅助Op
zeros = tf.zeros([2, 2])
ones = tf.ones([6])
uniform = tf.random_uniform([3, 3, 3], minval=0, maxval=10)  # 服从0~10的均匀分布
normal = tf.random_normal([3, 3, 2], mean=0.0, stddev=2.0)
trunc = tf.truncated_normal([2, 2], mean=5.0, stddev=1.0)   # 在2倍标准差范围内

# 2.初始化variables
init = tf.global_variables_initializer()
sess.run(init)


# 3.variable对象的修改
var_update = var1.assign(var1*2)
sess.run(var_update)
sess.run(var1.assign_add(1))
sess.run(var1.assign_sub(1))

6

In [14]:
# 名称作用域
with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name='A_add')
    b = a*3
with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = c*6
e = tf.add(b, d, name="output")
sess = tf.Session()
output = sess.run(e)
writer = tf.summary.FileWriter('./name_scope1', graph=tf.get_default_graph())
writer.close()

In [2]:
import tensorflow as tf
import numpy as np
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope("variables"):
        # 追踪数据流图的运行次数
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        # 追踪该模型的所有输出随时间的累加和的Variable
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")
    
    # 主要的变换op
    with tf.name_scope("transformation"):
        # 独立的输入层
        with tf.name_scope("input"):
            # 创建可接收一个向量的占位符
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
        # 独立的中间层
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
        # 独立的输出层
        with tf.name_scope("output"):
            output = tf.add(b, c, name='output')
    
    with tf.name_scope("update"):
        # 用最新的输出更新Variable对象total_output
        update_total = total_output.assign_add(output)
        # 计数器global_step +1
        increment_step = global_step.assign_add(1)
    
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        # 为输出节点创建汇总数据
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum_of_outputs', update_total)
        tf.summary.scalar('average_of_outputs', avg)
    with tf.name_scope("global_ops"):
        init = tf.global_variables_initializer()
        merged_summaries = tf.summary.merge_all()
        
# 运行data flow
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./frame', graph)

with graph.as_default():
    sess.run(tf.global_variables_initializer())


def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

for i in range(10):
    test_input = np.random.randint(0, 20, 5)
    run_graph(test_input)
    
writer.add_graph(graph, 10)

writer.flush()      # 数据填充完毕后，用flush()函数将汇总数据写入磁盘
writer.close()
sess.close()